In [ ]:
# section 7
# analogy task
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.models import KeyedVectors

import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
info = api.info()  # show info about available models/datasets
corpus = api.load('text8')
model1 = Word2Vec(corpus)

model = KeyedVectors.load_word2vec_format(datapath("C:/naheas/classes/NLP_CS5740/project/NLP-HW/google_wordvec/GoogleNews-vectors-negative300.bin"), binary=True)
# we can change models between google news and text8
# depending on use of model or model1
from collections import Counter, defaultdict
import numpy as np
file_path="C:/naheas/classes/NLP_CS5740/project/analogy_test.txt"
Fstream = open(file_path,'r')
Speech = Fstream.read()
lines = Speech.split('\n')
ans=0.0
not_ans=0
for i in lines:
    o=i.split()
    try:
        output=model.most_similar(positive=[o[2], o[1]], negative=[o[0]])
        if o[3]==(output[0])[0]:
            ans=ans+1
        else:
            print str(o) + " " + (output[0])[0]
            not_ans=not_ans+1
    except:
        not_ans=not_ans+1
print (ans/len(lines))

In [ ]:
#section 7
# new analogy task
print model.doesnt_match('mexican salsa nachos wireless'.split())
print model.doesnt_match('korean television kimchi seoul'.split())
print model.doesnt_match('car truck bus driver'.split())

oo = 'work working act acting'.split()
print str(oo) + " prediction: " + str(model.most_similar(positive=[oo[2], oo[1]], negative=[oo[0]])[0])
 
oo = 'do doing have having'.split()
print str(oo) + " prediction: " + str(model.most_similar(positive=[oo[2], oo[1]], negative=[oo[0]])[0])

oo = 'sleep sleeping go going'.split()
print str(oo) + " prediction: " + str(model.most_similar(positive=[oo[2], oo[1]], negative=[oo[0]])[0])

In [ ]:
# section 7
# analogy task 3
o = model.most_similar('increase')
for i in o:
    print str((i)[0]) + " " + str((i)[1])
o = model.most_similar('enter')
for i in o:
    print str((i)[0]) + " " + str((i)[1])
o = model.most_similar('gain')
for i in o:
    print str((i)[0]) + " " + str((i)[1])
o = model.most_similar('make')
for i in o:
    print str((i)[0]) + " " + str((i)[1])
o = model.most_similar('give')
for i in o:
    print str((i)[0]) + " " + str((i)[1])    


In [ ]:
#section 8
from collections import Counter, defaultdict
import numpy as np
import NGram
import math
from sklearn.metrics.pairwise import cosine_similarity as dist
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(datapath("C:/naheas/classes/NLP_CS5740/project/NLP-HW/google_wordvec/GoogleNews-vectors-negative300.bin"), binary=True)

model1=model

def Get_Vector(Paragraph):
    '''
    Input a paragraph, return log probability calculated on given Bigram
    '''
    p_vec = np.zeros([300,])
    cnt = 1
    for token in Paragraph:
        if token in model1:
            cnt += 1
            p_vec += model1[token]
    return p_vec/float(cnt)

def Predict_Ngram(Inpath = "../test/test.txt", Outpath = "../Output/8_1.csv", Train_Trump = "../train/trump.txt", Train_Obama = "../train/obama.txt"):
    '''
    Input:
        Inpath  : file path for test data
        Outpath : file path for output csv file
        Train_Trump : file path to train Trump's bigram model
        Train_Obama : file path to train Obama's bigram model
    Output:
        Return None, Output should go straight to .csv file
    '''
    f = open(Outpath, 'w')
    f.write('Id,Prediction\n')
    #Preprocess the test set
    Paragraphs_Trump = NGram.corpora_preprocess(Train_Trump)
    P_Vecs_Trump = [Get_Vector(p) for p in Paragraphs_Trump]
    Paragraphs_Obama = NGram.corpora_preprocess(Train_Obama)
    P_Vecs_Obama = [Get_Vector(p) for p in Paragraphs_Obama]
    Paragraphs_test = NGram.corpora_preprocess(Inpath)
    P_Vecs_test = [Get_Vector(p) for p in Paragraphs_test]
    for idx, pvec in enumerate(P_Vecs_test):
        max_cosine = -50
        isTrump = True
        for pv_trump in P_Vecs_Trump:
            max_cosine = max(max_cosine, dist([pvec], [pv_trump]))
        for pv_obama in P_Vecs_Obama:
            if dist([pvec], [pv_obama])>max_cosine:
                isTrump = False
                break
        f.write(str(idx)+',')
        if isTrump:
            f.write('1')
        else:
            f.write('0')
        f.write('\n')

def Preprocess(Train_Trump = "../train/trump.txt", Train_Obama = "../train/obama.txt"):
    Paragraphs_Trump = NGram.corpora_preprocess(Train_Trump)
    P_Vecs_Trump = np.array([Get_Vector(p) for p in Paragraphs_Trump[:-1]])
    Paragraphs_Obama = NGram.corpora_preprocess(Train_Obama)
    P_Vecs_Obama = np.array([Get_Vector(p) for p in Paragraphs_Obama[:-1]])
    X = np.concatenate((P_Vecs_Trump,P_Vecs_Obama))
    Y = np.concatenate((np.ones([3000,]),np.zeros([3000,])))
    return X,Y

def PreprocessFullTrain(Train_Trump = "../full_train/trump.txt", Train_Obama = "../full_train/obama.txt"):
    Paragraphs_Trump = NGram.corpora_preprocess(Train_Trump)
    P_Vecs_Trump = np.array([Get_Vector(p) for p in Paragraphs_Trump[:-1]])
    Paragraphs_Obama = NGram.corpora_preprocess(Train_Obama)
    P_Vecs_Obama = np.array([Get_Vector(p) for p in Paragraphs_Obama[:-1]])
    X = np.concatenate((P_Vecs_Trump,P_Vecs_Obama))
    Y = np.concatenate((np.ones([3100,]),np.zeros([3100,])))
    return X,Y

def Train_SGD(Outpath = '../Output/sgd.csv'):
    clf = SGDClassifier(loss="log", penalty="elasticnet")
    Paragraphs_test = NGram.corpora_preprocess("../test/test.txt")
    P_Vecs_test = np.array([Get_Vector(p) for p in Paragraphs_test[:-1]])
    X,Y = PreprocessFullTrain()
    clf.fit(X,Y)
    results = clf.predict(P_Vecs_test)
    Write_Result(Outpath, results)
    
def Train_SGDValidation(Outpath = '../Output/sgd.csv'):
    clf = SGDClassifier(loss="log", penalty="elasticnet")
    Paragraphs_test = NGram.corpora_preprocess("../development/obama.txt")
    P_Vecs_test = np.array([Get_Vector(p) for p in Paragraphs_test[:-1]])
    X,Y = Preprocess()
    clf.fit(X,Y)
    results = clf.predict(P_Vecs_test)
    
    cnt_obama = 0;
    for idx,label in enumerate(results):
        if label == 0:
            cnt_obama += 1
    print cnt_obama
    
    clf = SGDClassifier(loss="log", penalty="elasticnet")
    Paragraphs_test = NGram.corpora_preprocess("../development/trump.txt")
    P_Vecs_test = np.array([Get_Vector(p) for p in Paragraphs_test[:-1]])
    X,Y = Preprocess()
    clf.fit(X,Y)
    results = clf.predict(P_Vecs_test)
    
    cnt_trump = 0;
    for idx,label in enumerate(results):
        if label == 1:
            cnt_trump += 1
    print cnt_trump
    
    print str((cnt_obama + cnt_trump) / float(2))
    

def Train_RandomForest(Outpath = '../Output/forest2.csv', dp = 50, n = 100):
    clf = RandomForestClassifier(max_depth=dp, random_state=0, n_estimators=n, criterion='gini')
    Paragraphs_test = NGram.corpora_preprocess("../test/test.txt")
    P_Vecs_test = np.array([Get_Vector(p) for p in Paragraphs_test[:-1]])
    X,Y = PreprocessFullTrain()
    clf.fit(X,Y)
    results = clf.predict(P_Vecs_test)    
    Write_Result(Outpath, results)
    

def Train_RandomForestValidation(Outpath = '../Output/forest2.csv', dp = 50, n = 100):
    clf = RandomForestClassifier(max_depth=dp, random_state=0, n_estimators=n, criterion='gini')
    Paragraphs_test = NGram.corpora_preprocess("../development/obama.txt")
    P_Vecs_test = np.array([Get_Vector(p) for p in Paragraphs_test[:-1]])
    X,Y = Preprocess()
    clf.fit(X,Y)
    results = clf.predict(P_Vecs_test)
    
    cnt_obama = 0;
    for idx,label in enumerate(results):
        if label == 0:
            cnt_obama += 1
    print cnt_obama
    
    clf = RandomForestClassifier(max_depth=dp, random_state=0, n_estimators=n, criterion='gini')
    Paragraphs_test = NGram.corpora_preprocess("../development/trump.txt")
    P_Vecs_test = np.array([Get_Vector(p) for p in Paragraphs_test[:-1]])
    X,Y = Preprocess()
    clf.fit(X,Y)
    results = clf.predict(P_Vecs_test)
    
    cnt_trump = 0;
    for idx,label in enumerate(results):
        if label == 1:
            cnt_trump += 1
    print cnt_trump
    
    print str((cnt_obama + cnt_trump) / float(2))

def Write_Result(Outpath, results):
    f = open(Outpath, 'w')
    f.write('Id,Prediction\n')
    for idx,label in enumerate(results):
        f.write(str(idx)+',')
        if label == 1:
            f.write('1')
        else:
            f.write('0')
        f.write('\n')


In [105]:
Train_RandomForestValidation(dp=500,n=1000)

89
84
86.5


In [ ]:
Train_RandomForest(dp=1000)

In [63]:
Train_SGD()

In [74]:
Train_SGDValidation()

94
88
91
